In [28]:
# Install required packages
!pip install sqlalchemy==1.3.9 > nul 2>&1
!pip install ipython-sql > nul 2>&1
!pip install ipython-sql prettytable > nul 2>&1
!pip install -q pandas > nul 2>&1

In [29]:
# Load SQL extension
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [30]:
# Database connection
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

%sql sqlite:///my_data1.db

In [31]:
# Load dataset
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

DatabaseError: Execution failed on sql 'DROP TABLE "SPACEXTBL"': database is locked

In [32]:
# Remove blank rows from table
%sql DROP TABLE IF EXISTS SPACEXTABLE;
%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.


[]

In [33]:
# (Extra) Create missions table used in your queries
df.to_sql("missions", con, if_exists="replace", index=False)

101

In [34]:
# Preview
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [35]:
# Names of the unique launch sites in the space mission
df['Launch_Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [36]:
%%sql
SELECT DISTINCT Launch_Site
FROM missions;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [37]:
# 5 records where launch sites begin with the string 'CCA'
df[df['Launch_Site'].str.startswith('CCA', na=False)].head(5)

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [38]:
%%sql
SELECT *
FROM missions
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [39]:
# Total payload mass carried by boosters launched by NASA (CRS)
df.loc[
    df['Customer'].str.contains('NASA (CRS)', na=False, regex=False),
    'PAYLOAD_MASS__KG_'
].sum()

np.int64(48213)

In [40]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
FROM missions
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


Total_Payload_Mass
45596


In [41]:
# Average payload mass carried by booster version F9 v1.1
df.loc[df['Booster_Version'] == 'F9 v1.1', 'PAYLOAD_MASS__KG_'].mean()

np.float64(2928.4)

In [42]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS Avg_Payload_Mass
FROM missions
WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Avg_Payload_Mass
2928.4


In [43]:
# The date when the first succesful landing outcome in ground pad was achieved
df.loc[df['Landing_Outcome'] == 'Success (ground pad)', 'Date'].min()
df['Landing_Outcome'].unique()

array(['Failure (parachute)', 'No attempt', 'Uncontrolled (ocean)',
       'Controlled (ocean)', 'Failure (drone ship)',
       'Precluded (drone ship)', 'Success (ground pad)',
       'Success (drone ship)', 'Success', 'Failure', 'No attempt '],
      dtype=object)

In [44]:
%%sql
SELECT MIN(Date) AS First_Successful_Ground_Pad_Landing
FROM missions
WHERE Landing_Outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


First_Successful_Ground_Pad_Landing
2015-12-22


In [45]:
# The names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
df.loc[
    (df["Landing_Outcome"] == "Success (drone ship)") &
    (df["PAYLOAD_MASS__KG_"] > 4000) &
    (df["PAYLOAD_MASS__KG_"] < 6000),
    "Booster_Version"
].unique()

array(['F9 FT B1022', 'F9 FT B1026', 'F9 FT  B1021.2', 'F9 FT  B1031.2'],
      dtype=object)

In [46]:
%%sql
SELECT DISTINCT Booster_Version
FROM missions
WHERE Landing_Outcome = 'Success (drone ship)'
  AND PAYLOAD_MASS__KG_ > 4000
  AND PAYLOAD_MASS__KG_ < 6000;

%%sql
SELECT COUNT(DISTINCT Booster_Version) AS Distinct_Booster_Count
FROM missions;

 * sqlite:///my_data1.db
Done.
(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql
SELECT COUNT(DISTINCT Booster_Version) AS Distinct_Booster_Count
FROM missions;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [47]:
# The total number of successful and failure mission outcomes
is_success = df['Mission_Outcome'].str.contains('Success', case=False, na=False)
success_count = is_success.sum()
failure_count = (~is_success).sum()

print(success_count)
print(failure_count)

100
1


In [48]:
%%sql
SELECT Mission_Outcome, COUNT(*) AS Total
FROM missions
GROUP BY Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Total
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [49]:
# All the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function
max_mass = df['PAYLOAD_MASS__KG_'].max()
df.loc[df['PAYLOAD_MASS__KG_'] == max_mass, 'Booster_Version'].unique()

array(['F9 B5 B1048.4', 'F9 B5 B1049.4', 'F9 B5 B1051.3', 'F9 B5 B1056.4',
       'F9 B5 B1048.5', 'F9 B5 B1051.4', 'F9 B5 B1049.5',
       'F9 B5 B1060.2 ', 'F9 B5 B1058.3 ', 'F9 B5 B1051.6',
       'F9 B5 B1060.3', 'F9 B5 B1049.7 '], dtype=object)

In [50]:
%%sql
SELECT DISTINCT Booster_Version
FROM missions
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM missions
);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [51]:
# The records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

mask_2015 = df['Date'].dt.year == 2015
mask_fail_asds = df['Landing_Outcome'].str.contains('Failure (drone ship)', case=False, na=False, regex=False)

out = df.loc[mask_2015 & mask_fail_asds, ['Date', 'Landing_Outcome', 'Booster_Version', 'Launch_Site']].copy()
out['Month'] = out['Date'].dt.month_name()

out[['Month', 'Landing_Outcome', 'Booster_Version', 'Launch_Site']]

,Month,Landing_Outcome,Booster_Version,Launch_Site
13,January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
16,April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [52]:
%%sql
SELECT 
    strftime('%m', Date) AS Month_Number,
    CASE strftime('%m', Date)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS Month_Name,
    Booster_Version,
    Launch_Site,
    Landing_Outcome
FROM missions
WHERE strftime('%Y', Date) = '2015'
  AND Landing_Outcome LIKE 'Failure (drone ship)%';

 * sqlite:///my_data1.db
Done.


Month_Number,Month_Name,Booster_Version,Launch_Site,Landing_Outcome
01,January,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,April,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


In [53]:
# Ranking the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
mask_date = (df['Date'] >= '2010-06-04') & (df['Date'] <= '2017-03-20')

df.loc[mask_date, 'Landing_Outcome'].value_counts().sort_values(ascending=False)

Landing_Outcome
No attempt                10
Failure (drone ship)       5
Success (drone ship)       5
Controlled (ocean)         3
Success (ground pad)       3
Failure (parachute)        2
Uncontrolled (ocean)       2
Precluded (drone ship)     1
Name: count, dtype: int64